In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [2]:
dfe = pd.read_csv("employees.csv")

In [3]:
dfe.head()

,employe_id,branch_id,salary,join_date,resign_date
0,1,3,7500000,2018-08-23,NaN
1,7,1,7500000,2017-04-28,NaN
2,8,1,13000000,2017-04-28,NaN
3,9,1,13500000,2017-12-22,2020-10-14
4,10,1,9000000,2018-03-01,2020-12-20


In [4]:
dfe = dfe.rename(columns={'employe_id': 'employee_id'})

In [5]:
dfe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   employee_id  177 non-null    int64 
 1   branch_id    177 non-null    int64 
 2   salary       177 non-null    int64 
 3   join_date    177 non-null    object
 4   resign_date  39 non-null     object
dtypes: int64(3), object(2)
memory usage: 7.0+ KB


In [6]:
dfe['join_date'] = pd.to_datetime(dfe['join_date'])

In [7]:
dfe['resign_date'] = pd.to_datetime(dfe['resign_date'])

In [8]:
dfe['resign_date'] = dfe['resign_date'].fillna(datetime.today().strftime('%Y-%m-%d'))

In [9]:
dfe['month_interval'] = ((dfe.resign_date - dfe.join_date)/np.timedelta64(1, 'M')).apply(lambda x:math.floor(x))

In [10]:
dfe['total_salary'] = dfe.salary * dfe.month_interval

In [11]:
dfe_acc = dfe.groupby(['branch_id'])['total_salary'].sum().reset_index()

In [12]:
dfe_acc

,branch_id,total_salary
0,1,16706500000
1,2,5355500000
2,3,2033000000
3,2590,3711500000
4,2623,6803000000
5,2625,2215000000
6,2626,3011000000
7,2629,4924500000
8,2630,2808000000
9,2631,2039000000


In [13]:
dft = pd.read_csv("timesheets.csv")

In [14]:
dft = pd.merge(dft,dfe[['employee_id','branch_id']], on='employee_id',how='left')

In [15]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39792 entries, 0 to 39791
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   timesheet_id  39792 non-null  int64 
 1   employee_id   39792 non-null  int64 
 2   date          39792 non-null  object
 3   checkin       39065 non-null  object
 4   checkout      38437 non-null  object
 5   branch_id     39792 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.1+ MB


In [16]:
dft['date'] = pd.to_datetime(dft['date'])

In [17]:
dft['checkin'] = pd.to_datetime(dft['checkin'], format = '%H:%M:%S')

In [18]:
dft['checkout'] = pd.to_datetime(dft['checkout'], format = '%H:%M:%S')

In [19]:
dft['hours_interval'] = (dft.checkout - dft.checkin) / pd.Timedelta(hours=1)

In [22]:
dft_acc = dft.groupby(['branch_id'])['hours_interval'].sum().reset_index()

In [23]:
dft_acc

,branch_id,hours_interval
0,1,99846.160556
1,2,21927.865278
2,3,10940.703611
3,2590,21885.568333
4,2623,37489.253056
5,2625,12030.513611
6,2626,16484.227778
7,2629,23464.822778
8,2630,12877.632778
9,2631,13029.511111


In [24]:
df = pd.merge(dfe_acc,dft_acc, on='branch_id',how='inner')

In [28]:
df['salary_per_hour'] = round(df.total_salary / df.hours_interval)

In [29]:
df_final = df[['branch_id','salary_per_hour']]

In [30]:
df_final

,branch_id,salary_per_hour
0,1,167322.0
1,2,244233.0
2,3,185820.0
3,2590,169587.0
4,2623,181465.0
5,2625,184115.0
6,2626,182659.0
7,2629,209867.0
8,2630,218052.0
9,2631,156491.0
